# Get data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
#normalized train set, we need a normalized test set as well?!
normdata = pd.read_csv('../data/normalized_trainingdata.csv')

# data set not yet normalized
train_set = pd.read_csv('../data/train_set.csv')
test_set = pd.read_csv('../data/test_set.csv')

# Simple SVM

In [3]:
from sklearn.svm import SVC

# get x and y train
x_train = train_set[train_set.columns.drop('diagnosis')]
y_train = train_set['diagnosis']

# get x and y test
x_test = test_set[test_set.columns.drop('diagnosis')]
y_test = test_set['diagnosis']



In [ ]:
from sklearn.metrics import accuracy_score

SVM = SVC(kernel='linear')
SVM.fit(x_train, y_train)


y_predicted = linear.predict(x_test)
accuracy_score(y_test, y_predicted)

# Plot ROC curve SVM

In [ ]:
from sklearn.metrics import roc_curve, auc

# The linear classifier doesn't produce class probabilities by default. We'll retrain it for probabilities.
SVM_linear = SVC(kernel='linear', probability=True)
SVM_linear.fit(x_train, y_train)

# We'll need class probabilities from each of the classifiers
y_linear = SVM_linear.predict_proba(x_test)

# Compute the points on the curve
# We pass the probability of the second class (KIA) as the y_score
curve_linear = sklearn.metrics.roc_curve(y_test, y_linear[:, 1])

# Compute Area Under the Curve
auc_linear = auc(curve_linear[0], curve_linear[1])

plt.plot(curve_linear[0], curve_linear[1], label='linear (area = %0.2f)' % auc_linear)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve');

plt.legend();

# Crossvalidation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer

# The cross_val_score function does all the training for us. We simply pass 
# it the complete data, the model, and the metric.

SVM_linear = SVC(kernel='linear', probability=True)

# Train for 5 folds, returing ROC AUC. You can also try 'accuracy' as a scorer
scores = cross_val_score(linear, x, y, cv=3, scoring='roc_auc')

print('scores per fold ', scores)
print('  mean score    ', np.mean(scores))
print('  standard dev. ', np.std(scores))